In [5]:
#Importing the necessary libraries

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [7]:
url = 'https://www.daraz.pk/'

In [8]:
driver.get(url)

In [9]:
search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Redmi Note 12' + Keys.RETURN)

In [10]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

product_prices = []
delivery_statuses = []
ratings = []
seller_ratings = []
shipping_performances = []
flagship_statuses = []
scores = []
product_names = []


def find_element(xpath, parent=None):
    if parent:
        return parent.find_element(By.XPATH, xpath)
    return driver.find_element(By.XPATH, xpath)


def find_elements(xpath, parent=None):
    if parent:
        return parent.find_elements(By.XPATH, xpath)
    return driver.find_elements(By.XPATH, xpath)


def scroll_down(height):
    driver.execute_script(f"window.scrollTo(0, {height});")
    time.sleep(1)


def scrape_product(index):
    current_card = ad_cards[index]
    current_card.click()
    information_grid = find_element('/html/body/div[4]/div/div[3]')
    product_prices.append(
        find_element('//*[@id="module_product_price_1"]/div/div/span', information_grid).text
    )
    delivery_bar = find_element('//*[@id="block-Cve5yP08rW9"]', information_grid)
    delivery_opt = find_elements('.//*[contains(@class, "delivery-option-item__title")]', delivery_bar)
    seller_rating = find_element(
        '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]', delivery_bar
    ).text
    shipping_perf = find_element('//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]', delivery_bar).text
    flagship_mall_element = find_elements(
        '//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[2]/img', delivery_bar
    )
    name_of_product = find_element(
        '//*[@id="module_product_title_1"]/div/div', information_grid
    ).text
    product_names.append(name_of_product)
    flagship_statuses.append(1) if flagship_mall_element else flagship_statuses.append(0)
    seller_ratings.append(seller_rating)
    shipping_performances.append(shipping_perf)
    scroll_down(500)
    scroll_down(800)
    scroll_down(1000)
    scroll_down(1200)
    rating_bar = find_element('//*[@id="module_product_review"]', information_grid)
    rating = rating_bar.find_element(
        By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div/div[1]/div[2]/div/div/div[1]/div[1]/span[1]'
    ).text
    ratings.append(rating)
    counter = sum('Free Delivery' in i.text for i in delivery_opt)
    delivery_statuses.append(1) if counter > 0 else delivery_statuses.append(0)
    print(index)
    scores.append(calculate_score(index))
    time.sleep(2)
    driver.back()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]'))
    )


def calculate_score(product_index):
    try:
        base_score = int(product_prices[product_index].replace('Rs. ', '').replace(',', ''))
    except (ValueError, IndexError):
        base_score = 0

    daraz_mall_score = 5000 if flagship_statuses[product_index] == 1 else 0
    free_shipping_score = 1000 if delivery_statuses[product_index] == 1 else 0

    try:
        product_rating = float(ratings[product_index])
    except (ValueError, IndexError):
        product_rating = 0.0

    try:
        seller_rating = float(seller_ratings[product_index].replace('%', ''))
    except (ValueError, IndexError):
        seller_rating = 0.0

    try:
        shipping_performance = float(shipping_performances[product_index].replace('%', ''))
    except (ValueError, IndexError):
        shipping_performance = 0.0

    return base_score + daraz_mall_score + free_shipping_score + 10000 * (
        (0.4 * (product_rating / 5)) + (0.4 * (seller_rating / 100)) + (0.2 * (shipping_performance / 100))
    )


# Set the range of elements you want to process
start_index = 0  # You can change this to the desired starting index
end_index = 3  # You can change this to the desired ending index

#driver.back()
for index in range(start_index, end_index):
    driver.execute_script(f"window.scrollTo(0, 100);")
    ad_grid = find_element('//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')

    # Scroll down and wait for new elements to load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Adjust the sleep duration based on the page loading time

    ad_cards = find_elements('.//*[contains(@class, "gridItem--Yd0sa")]', ad_grid)

    # Check if the index is within the valid range
    if index < len(ad_cards):
        scrape_product(index)
        # Reinitialize ad_cards after going back
        ad_grid = find_element('//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
        ad_cards = find_elements('.//*[contains(@class, "gridItem--Yd0sa")]', ad_grid)
    else:
        print(f"Skipping index {index} as it is out of range.")

best_product_index = scores.index(max(scores))

# Print the details of the best product
print(f"Best Product Details:")
print(f"Name: {product_names[best_product_index]}")
print(f"Price: {product_prices[best_product_index]}")
print(f"Daraz Mall Status: {'Yes' if flagship_statuses[best_product_index] == 1 else 'No'}")
print(f"Free Shipping: {'Yes' if delivery_statuses[best_product_index] == 1 else 'No'}")
print(f"Product Rating: {ratings[best_product_index]}")
print(f"Seller Rating: {seller_ratings[best_product_index]}")
print(f"Shipping Performance: {shipping_performances[best_product_index]}")
print(f"Total Score: {scores[best_product_index]}")


0
1
2
Best Product Details:
Name: Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB Front Camera 16 MP Back Camera 108 MP + 08 MP + 02 MP Battery Type Li-Po 5000 mAh, non-removable
Price: Rs. 88,840
Daraz Mall Status: No
Free Shipping: Yes
Product Rating: 5.0
Seller Rating: 93%
Shipping Performance: 100%
Total Score: 99560.0


In [12]:
ad_cards[best_product_index].click()


In [13]:
import time
import pandas as pd

temp = dict()
temp['Name'] = []
temp['Review text'] = []

driver.execute_script("window.scrollTo(0, 500);")
time.sleep(1)
driver.execute_script("window.scrollTo(500, 800);")
time.sleep(1)
driver.execute_script("window.scrollTo(800, 1000);")
time.sleep(2)
driver.execute_script("window.scrollTo(1000, 1300);")
time.sleep(2)
driver.execute_script("window.scrollTo(1300, 1600);")
time.sleep(1)

for page in range(0,4):
    try:
        reviews_container = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[1]')
    except NoSuchElementException:
        print('No Reviews')
        break
    reviews = reviews_container.find_elements(By.CLASS_NAME,'item')

    for i in range(len(reviews)):

        if reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text != '':
            temp['Name'].append(reviews[i].find_element(By.CLASS_NAME,'middle').find_element(By.TAG_NAME,'span').text[3:])
            temp['Review text'].append(reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text)

    try:
        if(page != 4):
            nxt_button = driver.find_element(By.XPATH, f'//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/div/button[2]')
            driver.execute_script("arguments[0].click();", nxt_button)
            time.sleep(5)
    except NoSuchElementException:
        print('No More Pages')
        break

reviews = pd.DataFrame(temp)

reviews

,Name,Review text
0,Talal F.,waited for Redmi Note 12 Pro's release at Xiao...
1,Kashif A.,"100 % Genuine Product , Too fast delivery by s..."
2,irfan R.,"i ordered Black one, but i got white one which..."
3,3***3,Brand new packed phone. Delivery was a bit slo...
4,Muhammad N.,Very Good response from seller and received wi...
5,Muhammad A.,Excellent Product and very fast delivery.
6,Haider J.,Super Sonic fast Delivery from Seller. I just ...
7,Dr M.,"👍👍👍Fast delivery, Seller is Recommended 👍👍👍"
8,*******904,Excellent product original received and timely...
9,Syed M.,Fastest Delivery and genuine product. This sel...


In [14]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
import pandas as pd
from textblob import TextBlob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
# Your existing code for scraping reviews
# ...

# Assuming you have a DataFrame called 'reviews' with 'Name' and 'Review text' columns

# Initialize the SentimentIntensityAnalyzer
# sia = SentimentIntensityAnalyzer()

# # Analyze sentiment for each review and add a new column 'Sentiment' to the DataFrame
# reviews['Sentiment'] = reviews['Review text'].apply(lambda x: sia.polarity_scores(x)['compound'])

# # Classify sentiments as positive, neutral, or negative based on the compound score
# reviews['Sentiment'] = reviews['Sentiment'].apply(lambda x: 'Positive' if x > 0.05 else ('Negative' if x < -0.05 else 'Neutral'))

# # Display the DataFrame with sentiments
# print(reviews)

# # Save the DataFrame with sentiments to a CSV file
# reviews.to_csv('Reviews_with_Sentiments.csv', index=False)
def get_sentiment(text):
    analysis = TextBlob(text)
    # Use the polarity to determine sentiment
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Add a new 'Sentiment' column to the DataFrame
reviews['Sentiment'] = reviews['Review text'].apply(get_sentiment)

# Display the DataFrame with sentiments
print(reviews)

# Save the DataFrame with sentiments to a CSV file
reviews.to_csv('Reviews_with_Sentiments.csv', index=False)

           Name                                        Review text Sentiment
0      Talal F.  waited for Redmi Note 12 Pro's release at Xiao...  Positive
1     Kashif A.  100 % Genuine Product , Too fast delivery by s...  Positive
2      irfan R.  i ordered Black one, but i got white one which...  Positive
3         3***3  Brand new packed phone. Delivery was a bit slo...  Positive
4   Muhammad N.  Very Good response from seller and received wi...  Positive
5   Muhammad A.          Excellent Product and very fast delivery.  Positive
6     Haider J.  Super Sonic fast Delivery from Seller. I just ...  Positive
7         Dr M.        👍👍👍Fast delivery, Seller is Recommended 👍👍👍   Neutral
8    *******904  Excellent product original received and timely...  Positive
9       Syed M.  Fastest Delivery and genuine product. This sel...  Positive
10  Muhammad A.          Excellent Product and very fast delivery.  Positive
11    Haider J.  Super Sonic fast Delivery from Seller. I just ...  Positive

In [22]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

# Your existing code for scraping reviews
# ...

# Assuming you have a DataFrame called 'reviews' with 'Name' and 'Review text' columns

# Define a dictionary of positive words and emojis
positive_words = ['good', 'kind', 'positive', 'fast', 'excellent', 'amazing', 'awesome', 'fantastic', 'genuine', 'on time', 'original', 'zabrdast', 'geniun', 'well', 'Thank you', 'Thanks', 'satisfying', 'pleasing', 'superb', 'outstanding', 'impressive', 'stellar', 'remarkable', 'awesome', 'amazing']
negative_words = ['bad', 'poor', 'negative', 'slow', 'disappointing', 'horrible', 'awful', 'terrible', 'fake', 'not genuine', 'late', 'worst', 'unpleasant', 'problematic', 'displeased', 'regret']
positive_emojis = ['😊', '👍', '🌟', '😄', '🎉']
negative_emojis = ['😞', '👎', '💔', '😢', '😔', '😡', '🤬', '😤', '😠']

# Combine positive words and emojis into a single list
positive_lexicon = positive_words + positive_emojis
negative_lexicon = negative_words + negative_emojis
# Define a function to get sentiment using the custom positive lexicon

def get_sentiment_with_lexicon(text):
    # Check if any positive word or emoji is present in the text
    counter_positive_words = 0
    counter_negative_words = 0
    if any(word in text for word in positive_lexicon):
        counter_positive_words += 1
        
    if any(word in text for word in negative_words):  # Fixed the typo here
        counter_negative_words += 1
        
    if counter_negative_words < counter_positive_words:
        return 'Positive'
    elif counter_negative_words > counter_positive_words:
        return 'Negative'
    if(counter_positive_words==counter_negative_words):
        return 'Nuetral'
    else:
        return 'Neutral'  # Assume neutral if no positive word or emoji is found
# Add a new 'Sentiment' column to the DataFrame using the custom lexicon
reviews['Sentiment'] = reviews['Review text'].apply(get_sentiment_with_lexicon)

# Display the DataFrame with sentiments
print(reviews)

# Save the DataFrame with sentiments to a CSV file
reviews.to_csv('Reviews_with_Sentiments.csv', index=False)


           Name                                        Review text Sentiment
0      Talal F.  waited for Redmi Note 12 Pro's release at Xiao...  Positive
1     Kashif A.  100 % Genuine Product , Too fast delivery by s...  Positive
2      irfan R.  i ordered Black one, but i got white one which...  Positive
3         3***3  Brand new packed phone. Delivery was a bit slo...  Negative
4   Muhammad N.  Very Good response from seller and received wi...   Nuetral
5   Muhammad A.          Excellent Product and very fast delivery.  Positive
6     Haider J.  Super Sonic fast Delivery from Seller. I just ...  Positive
7         Dr M.        👍👍👍Fast delivery, Seller is Recommended 👍👍👍  Positive
8    *******904  Excellent product original received and timely...  Positive
9       Syed M.  Fastest Delivery and genuine product. This sel...  Positive
10  Muhammad A.          Excellent Product and very fast delivery.  Positive
11    Haider J.  Super Sonic fast Delivery from Seller. I just ...  Positive